In [ ]:
from astropy.table import Table
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import aplpy
import pyregion
from astropy.coordinates import SkyCoord
from sklearn import linear_model, datasets
import scipy
from astropy.table import Table
from scipy.optimize import curve_fit
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve
from scipy.integrate import trapz
from scipy import signal
import pandas as pd
from glob import glob
import os
from astropy.io import ascii

In [ ]:
all_stars = Table.read('../HST_Guido/30dor_all_newerr.UBVIHa.rot', format='ascii').to_pandas()


In [ ]:
all_stars_excess = Table.read('../HST_Guido/30dormptcoo_coords.txt', format='ascii').to_pandas()
#all_stars_excess.columns = 'ID;x;y;RA;Dec;u_1;eu_2;b_1;eb_2;v_1;ev_2;i_1;ei_2;ha_1;eha_2'.split(';')
all_stars_excess.columns = ['X', 'Y']

In [ ]:
SexTab_Ha = Table.read('Ha.cat', format="fits", hdu='LDAC_OBJECTS')
SexTab_V  = Table.read('V.cat', format="fits", hdu='LDAC_OBJECTS')
SexTab_I  = Table.read('I.cat', format="fits", hdu='LDAC_OBJECTS')

In [ ]:
SexTab_Ha = SexTab_Ha[(SexTab_Ha['MAG_AUTO']>-50)*(SexTab_Ha['MAG_AUTO']<50)]
SexTab_I  = SexTab_I[(SexTab_I['MAG_AUTO']>-50)*(SexTab_I['MAG_AUTO']<50)]
SexTab_V  = SexTab_V[(SexTab_V['MAG_AUTO']>-50)*(SexTab_V['MAG_AUTO']<50)]

sexVmags  = SexTab_V['MAG_ISOCOR']  -0.1  + 25.81
sexImags  = SexTab_I['MAG_ISOCOR']  -0.1  + 24.67
sexHamags = SexTab_Ha['MAG_ISOCOR'] -0.1  + 19.92

In [ ]:
phot_V  = ascii.read('V_filter.phot')
phot_I  = ascii.read('I_filter.phot')
phot_Ha = ascii.read('Ha_filter.phot')

irafVmags  = phot_V['MAG']  -0.1 + 25.81
irafImags  = phot_I['MAG']  -0.1 + 24.67
irafHamags = phot_Ha['MAG'] -0.1 + 19.92

In [ ]:
# Sextractor vs IRAF
plt.figure(figsize=(8,6))
plt.hist(all_stars.v_1, normed=True, bins=np.arange(15,28,0.5), histtype='step', linewidth=2, color='red', label='IRAF V')
plt.hist(sexVmags, normed=True, bins=np.arange(15,28,0.5), histtype='step', linewidth=2, color='red', linestyle='dotted', label='SEX V')

plt.hist(all_stars.i_1, normed=True, bins=np.arange(15,28,0.5), histtype='step',  color='blue',linewidth=2, label='IRAF I')
plt.hist(sexImags, normed=True, bins=np.arange(15,28,0.5), histtype='step', linewidth=2, color='blue', linestyle='dotted', label='SEX I')

plt.hist(all_stars.ha_1, normed=True, bins=np.arange(15,28,0.5), histtype='step', linewidth=2, color='green',label='IRAF Ha')
plt.hist(sexHamags, normed=True, bins=np.arange(15,28,0.5), histtype='step', linewidth=2, color='green', linestyle='dotted', label='SEX Ha')

#plt.hist(all_stars.v_1, bins=np.arange(17,28,0.5), normed=True)
plt.legend(loc='best')
plt.show()

In [ ]:
# Ha excess stars
phot_V_e  = ascii.read('V_filter_excess.phot')
phot_I_e  = ascii.read('I_filter_excess.phot')
phot_Ha_e = ascii.read('Ha_filter_excess.phot')

irafVmags_e  = phot_V_e['MAG']  -0.1 + 25.81
irafImags_e  = phot_I_e['MAG']  -0.1 + 24.67
irafHamags_e = phot_Ha_e['MAG'] -0.1 + 19.92

In [ ]:
# Lowest number of detections in Ha
mag_keyword = 'MAG'
for det in SexTab_Ha:
    ra, dec = det['XPEAK_IMAGE'], det['YPEAK_IMAGE']
    dI = np.sqrt(((ra-SexTab_I['XPEAK_IMAGE'])**2 +  (dec-SexTab_I['YPEAK_IMAGE'])**2))
    dV = np.sqrt(((ra-SexTab_V['XPEAK_IMAGE'])**2 +  (dec-SexTab_V['YPEAK_IMAGE'])**2))
    
    Hamag = det[mag_keyword] + mzp_Ha
    
    minI, minV = np.argmin(dI), np.argmin(dV)
    if (np.min(dI) < 12) * (np.min(dV)<12):
        Vmag = SexTab_V[minV][mag_keyword] + mzp_V
        Imag = SexTab_I[minI][mag_keyword] + mzp_I
        plt.scatter(Vmag-Imag, Vmag-Hamag, c='black', s=1)
#plt.xlim(-0.5,3)
#plt.ylim(-2,5)
plt.show()

In [ ]:
# Lowest number of detections in Ha
mag_keyword = 'MAG_APER'
Vmags, VIcolors = [], []
for det in SexTab_V:
    ra, dec = det['XPEAK_IMAGE'], det['YPEAK_IMAGE']
    dI = np.sqrt(((ra-SexTab_I['XPEAK_IMAGE'])**2 +  (dec-SexTab_I['YPEAK_IMAGE'])**2))
    #dV = np.sqrt(((ra-SexTab_V['XPEAK_IMAGE'])**2 +  (dec-SexTab_V['YPEAK_IMAGE'])**2))
    
    minI = np.argmin(dI)
    if (np.min(dI) < 2):
        if det['MAGERR_APER'] + SexTab_I[minI]['MAGERR_APER'] < 0.07:
            Vmag = det[mag_keyword]+mzp_V
            Imag = SexTab_I[minI][mag_keyword] + mzp_I

            Vmags.append(Vmag)
            VIcolors.append(Vmag - Imag)
        #plt.scatter(SexTab_V[minV]['MAG_AUTO']-SexTab_I[minI]['MAG_AUTO'], SexTab_V[minV]['MAG_AUTO']+21, c='black', s=1)
#plt.xlim(-0.5,3)
#plt.ylim(-2,5)
#plt.show()

In [ ]:
plt.scatter(VIcolors, Vmags, s=1)
plt.xlim(-0.5,3.2)
plt.ylim(30,10)

In [ ]:
toplot = all_stars.ei_2 + all_stars.ev_2 <= 0.07
plt.scatter((all_stars.v_1-all_stars.i_1)[toplot], all_stars.v_1[toplot], s=0.3, c='grey')
plt.xlim(-0.5,3.2)
plt.ylim(30,10)
plt.show()

In [ ]:
plt.hist(SexTab_I['MAG_APER']+mzp_I, bins=np.arange(-5,5,0.5)+mzp_I, normed=True)
plt.hist(SexTab_V['MAG_APER']+mzp_V, bins=np.arange(-5,5,0.5)+mzp_V, normed=True)
plt.hist(all_stars.v_1, bins=np.arange(17,28,0.5), normed=True)

In [ ]:
all_stars[['x', 'y']].to_csv('all_stars_xy_coords_sex.coo', sep='\t', index=True)

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(irafVmags, normed=True, bins=np.arange(17,28,0.5), histtype='step', linewidth=2, color='red', label='Luuk V')
plt.hist(all_stars.v_1, normed=True, bins=np.arange(17,28,0.5), histtype='step', linewidth=2, color='red', linestyle='dotted', label='Guido V')

plt.hist(irafImags, normed=True, bins=np.arange(17,28,0.5), histtype='step',  color='blue',linewidth=2, label='Luuk I')
plt.hist(all_stars.i_1, normed=True, bins=np.arange(17,28,0.5), histtype='step', linewidth=2, color='blue', linestyle='dotted', label='Guido I')

plt.hist(irafHamags, normed=True, bins=np.arange(17,28,0.5), histtype='step', linewidth=2, color='green',label='Luuk Ha')
plt.hist(all_stars.ha_1, normed=True, bins=np.arange(17,28,0.5), histtype='step', linewidth=2, color='green', linestyle='dotted', label='Guido Ha')

#plt.hist(all_stars.v_1, bins=np.arange(17,28,0.5), normed=True)
plt.legend(loc='best')
plt.xlabel('Magnitude')
plt.ylabel('Normalized denisity')
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(131)
plt.scatter(all_stars.i_1, irafImags, s=1)
plt.plot(np.arange(14,28),np.arange(14,28), c='r')
plt.xlim(12,28)
plt.xlabel('Guido mags')
plt.ylabel('Luuk mags')
plt.title('I band')

plt.subplot(132)
plt.scatter(all_stars.v_1, irafVmags, s=1)
plt.plot(np.arange(14,28),np.arange(14,28), c='r')
plt.xlim(12,28)
plt.xlabel('Guido mags')
plt.ylabel('Luuk mags')
plt.title('V band')

plt.subplot(133)
plt.scatter(all_stars.ha_1, irafHamags, s=1)
plt.plot(np.arange(14,28),np.arange(14,28), c='r')
plt.xlim(12,28)
plt.xlabel('Guido mags')
plt.ylabel('Luuk mags')
plt.tight_layout()
plt.title('Ha band')
plt.show()

In [ ]:
f = open('iraf_regions.reg', 'w')
f.write(
"""# Region file format: DS9 version 4.1
global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
image\n""")

In [ ]:
for i in range(len(phot_V)):
    f.write('circle('+str(phot_V['XCENTER'][i]) + ',' + str(phot_V['YCENTER'][i])+',3)\n')
    f.write('circle('+str(phot_V['XCENTER'][i]) + ',' + str(phot_V['YCENTER'][i])+',4)\n')
    f.write('circle('+str(phot_V['XCENTER'][i]) + ',' + str(phot_V['YCENTER'][i])+',7)\n')
f.close()

In [ ]:
plt.scatter((irafVmags-irafImags)[toplot], (irafVmags-irafHamags)[toplot], s=0.1, c='grey')
plt.scatter((irafVmags_e-irafImags_e), (irafVmags_e-irafHamags_e), s=2, c='red')
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 4.5)
theoretic_model = np.array([0.1,0.1,0.8,0.6, 1.3, 1, 3,2.15]).reshape(4,2)
plt.plot(theoretic_model[:,0], theoretic_model[:,1], linestyle='dotted', linewidth=4, c='black')

In [ ]:
from astroquery.mast import Observations

flist = glob('./Ha_var/ha*.fits')
tstart_dict = {}
for f in flist:
    filename = f.split('/')[-1][:-5]
    header = fits.open(f)[0].header
    obsid = header['EXPNAME']
    obs_table = Observations.query_criteria(obs_id=obsid)
    t_start = obs_table['t_min'].data[0]
    t_end = obs_table['t_max'].data[0]
    tstart_dict[filename] = t_start

In [ ]:
flist = glob('./Ha_var/*.phot')
df = pd.DataFrame(np.zeros((0,16)))

colnames = np.array(np.sort([w.split('/')[-1] for w in 2*flist]), dtype=str)
for i, name in enumerate(colnames):
    if i%2 == 1:
        colnames[i] = (name[:-5])
    if i%2 == 0:
        colnames[i] = (name[:-5] + '.err')
df.columns = colnames
for file in flist:
    var_phot_Ha = ascii.read(file)
    var_irafHamags = var_phot_Ha['MAG'] -0.1 + 19.92
    df[file.split('/')[-1][:-5]+'.err'] = var_phot_Ha['MERR']
    df[file.split('/')[-1][:-5]] = var_irafHamags
df[df==0] = np.nan
df['std'] = np.nanstd(df[['ha'+str(w) for w in np.arange(1,9)]].values, axis=1)
df['mean'] = np.nanmean(df[['ha'+str(w) for w in np.arange(1,9)]].values, axis=1)

In [ ]:
plt.figure(figsize=(8,5))
for i, row in df[(df['std']>0.5) * (df['mean']<19)* (df['mean']>15.5)].iterrows():
    mags    = (row[['ha'+str(w) for w in np.arange(1,9)]].values)
    magerrs = (row[['ha'+str(w)+'.err' for w in np.arange(1,9)]].values)
    timestamps = [tstart_dict[w] for w in ['ha'+str(w) for w in np.arange(1,9)]]
    timestamps -= np.min(timestamps)

    order = np.argsort(timestamps)
    timestamps = np.sort(timestamps)
    mags, magerrs = mags[order], magerrs[order]
    if np.max(magerrs) > 1:
        continue
    if np.sum(np.isnan(mags)) + np.sum(np.isnan(magerrs)) > 0:
        continue
    plt.errorbar(timestamps, mags, yerr=magerrs, c='black', linestyle='dotted')
    plt.scatter(timestamps, mags, c='red', s=10)
plt.ylabel('Ha magnitude')
plt.xlabel(r'$\Delta \, T$ [days]')
plt.show()   

In [ ]:
var_phot_Ha = ascii.read('./Ha_var/ha2.phot')

In [ ]:
TEXP = 1
flux = var_phot_Ha['FLUX']
error = np.sqrt(TEXP * var_phot_Ha['FLUX'] / 1.56 + var_phot_Ha['AREA']*(0.6*TEXP)**2+var_phot_Ha['AREA']**2 * (0.6*TEXP)**2/var_phot_Ha['NSKY'])

1.056 * error / flux

In [ ]:
### MAG offset for 3 pixel radius aperture vs 100 pixel radius aperture ###
### Esstimated for GD153 ####

In [ ]:
flux03 = ascii.read('../PSF_star/Ha_03.phot')
flux100 = var_phot_Ha = ascii.read('../PSF_star/Ha_100.phot')
flux100['MAG'] - flux03['MAG']

In [ ]:
flux03 = ascii.read('../PSF_star/V_03.phot')
flux100 = var_phot_Ha = ascii.read('../PSF_star/V_100.phot')
flux100['MAG'] - flux03['MAG']

In [ ]:
flux03 = ascii.read('../PSF_star/I_03.phot')
flux100 = var_phot_Ha = ascii.read('../PSF_star/I_100.phot')
flux100['MAG'] - flux03['MAG']

In [ ]:
phot_V1  = ascii.read('../IRAF_cats_drz/F555W/deep/drz_sci_0_astrometry_regrid.phot')['MAG']  -0.1 + 25.81
phot_V2  = ascii.read('../IRAF_cats_drz/F555W/deep/drz_sci_1_astrometry_regrid.phot')['MAG']  -0.1 + 25.81
phot_V3  = ascii.read('../IRAF_cats_drz/F555W/deep/drz_sci_2_astrometry_regrid.phot')['MAG']  -0.1 + 25.81
phot_V4  = ascii.read('../IRAF_cats_drz/F555W/deep/drz_sci_3_astrometry_regrid.phot')['MAG']  -0.1 + 25.81

In [ ]:
std_V  = np.std(np.vstack((phot_V1.data.data, phot_V2.data.data, phot_V3.data.data, phot_V4.data.data)), axis=0)
mag_v  = np.median(np.vstack((phot_V1.data.data, phot_V2.data.data, phot_V3.data.data, phot_V4.data.data)), axis=0)
to_use = (std_V<0.05)*mag_v>18
toplot = (all_stars.ev_2 <= 0.05)*(all_stars.v_1>18)

In [ ]:
phot_V1  = ascii.read('../Ha_PSF_photometry/ib6wd1rrq_flc_drz_sci_astrometry_regrid_exptime.phot')
#to_usephot_V1['MAG']-0.1 + 19.92


In [ ]:
plt.scatter(phot_V1['MAG']  -0.1 + 19.92, phot_V1['MERR'], s=1)
plt.ylim(0,10)